# Invoicing

Now that we have set up our user info, clients, contracts and projects, as well as a source for time tracking data, we are ready to automatically generate invoices.

## Preamble

In [1]:
from pathlib import Path
import datetime
from IPython import display

In [2]:
import tuttle

In [3]:
app = tuttle.app.App()

2022-02-20 13:56:09,050 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-20 13:56:09,051 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-02-20 13:56:09,051 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-20 13:56:09,051 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2022-02-20 13:56:09,051 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-20 13:56:09,052 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("icloudaccount")
2022-02-20 13:56:09,052 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-20 13:56:09,052 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("googleaccount")
2022-02-20 13:56:09,052 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-20 13:56:09,053 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("bank")
2022-02-20 13:56:09,053 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-20 13:56:09,053 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("bankaccount")
2022-02-20 13:56:09,054 INFO sqlalchemy.engine.Engine [r

## How it works

We assume that you want to create and send an invoice for a `Project`. In the invoice, you bill the client for work time - that is, one or several `Timesheet`s.

## Workflow

_1. Select a project_

**Example**

In [4]:
my_project = app.get_project(title="Heating Repair")

2022-02-20 13:56:09,062 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-20 13:56:09,074 INFO sqlalchemy.engine.Engine SELECT project.id, project.title, project.tag, project.start_date, project.end_date, project.contract_id 
FROM project 
WHERE project.title = ?
2022-02-20 13:56:09,074 INFO sqlalchemy.engine.Engine [no key 0.00068s] ('Heating Repair',)


/Users/cls/miniforge3/envs/tuttle/lib/python3.9/site-packages/sqlmodel/orm/session.py:60: SAWarning: Class SelectOfScalar will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  results = super().execute(


_Your turn:_

2. Select a time tracking data source.

**Example**

In [5]:
from tuttle.calendar import FileCalendar

timetracking_calendar_path = Path("../../tests/data/TuttleDemo-TimeTracking.ics")
my_calendar = FileCalendar(
    path=timetracking_calendar_path, 
    name="TimeTracking"
)

_Your turn:_

3. Generate one or more timesheets

**Example**

In [6]:
my_timesheet = tuttle.timetracking.generate_timesheet(
    source=my_calendar,
    project=my_project,
    period="February 2022",
    item_description=my_project.title,
)

2022-02-20 13:56:09,192 INFO sqlalchemy.engine.Engine SELECT contract.unit AS contract_unit, contract.billing_cycle AS contract_billing_cycle, contract.id AS contract_id, contract.title AS contract_title, contract.signature_date AS contract_signature_date, contract.start_date AS contract_start_date, contract.end_date AS contract_end_date, contract.client_id AS contract_client_id, contract.rate AS contract_rate, contract.currency AS contract_currency, contract.units_per_workday AS contract_units_per_workday, contract.volume AS contract_volume, contract.term_of_payment AS contract_term_of_payment 
FROM contract 
WHERE contract.id = ?
2022-02-20 13:56:09,192 INFO sqlalchemy.engine.Engine [generated in 0.00039s] (1,)


/Users/cls/miniforge3/envs/tuttle/lib/python3.9/site-packages/sqlmodel/orm/session.py:101: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  return super().execute(  # type: ignore


4. Generate an invoice for the timesheet(s).

**Example**

In [7]:
my_invoice = tuttle.invoicing.generate_invoice(
    timesheets=[
        my_timesheet,
    ],
    contract=my_project.contract,
    date=datetime.date.today(),
)

In [8]:
my_invoice.number

'2022-02-20-01'

In [9]:
my_invoice.total

Decimal('952.000000000000')

_Your turn_

5. Render the invoice to a document template:

You can display the HTML in the notebook...

In [10]:
display.HTML(
    tuttle.rendering.render_invoice(
        user=app.user, 
        invoice=my_invoice,
        style=None
    )
)

2022-02-20 13:56:09,213 INFO sqlalchemy.engine.Engine INSERT INTO invoice (number, date, contract_id, sent, paid, cancelled) VALUES (?, ?, ?, ?, ?, ?)
2022-02-20 13:56:09,213 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ('2022-02-20-01', '2022-02-20', 1, None, None, None)
2022-02-20 13:56:09,215 INFO sqlalchemy.engine.Engine INSERT INTO invoiceitem (date, quantity, unit, unit_price, description, "VAT_rate", invoice_id) VALUES (?, ?, ?, ?, ?, ?, ?)
2022-02-20 13:56:09,215 INFO sqlalchemy.engine.Engine [generated in 0.00029s] ('2022-02-20', 16, 'hour', 50.0, 'Heating Repair - February 2022', 0.19, 1)
2022-02-20 13:56:09,216 INFO sqlalchemy.engine.Engine INSERT INTO timesheet (title, date, period, project_id, comment) VALUES (?, ?, ?, ?, ?)
2022-02-20 13:56:09,217 INFO sqlalchemy.engine.Engine [generated in 0.00031s] ('Heating Repair - February 2022', '2022-02-20', 'February 2022', 1, '')
2022-02-20 13:56:09,217 INFO sqlalchemy.engine.Engine INSERT INTO timetrackingitem (timeshee

... or render it to a file:

_Set the path to a folder where you want your invoices to appear_:

In [11]:
invoice_dir = Path.home() / "Downloads"

In [12]:
tuttle.rendering.render_invoice(
    user=app.user, 
    invoice=my_invoice,
    style="Anvil",
    out_dir=invoice_dir,
)

2022-02-20 13:56:09,250 INFO sqlalchemy.engine.Engine SELECT user.id, user.name, user.subtitle, user.website, user.e_mail, user.phone_number, user.address_id, user."VAT_number", user.icloud_account_id, user.bank_account_id, user.logo 
FROM user
2022-02-20 13:56:09,250 INFO sqlalchemy.engine.Engine [no key 0.00051s] ()
2022-02-20 13:56:09,255 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.street AS address_street, address.number AS address_number, address.city AS address_city, address.postal_code AS address_postal_code, address.country AS address_country 
FROM address 
WHERE address.id = ?
2022-02-20 13:56:09,256 INFO sqlalchemy.engine.Engine [cached since 0.02651s ago] (1,)
2022-02-20 13:56:09,257 INFO sqlalchemy.engine.Engine SELECT bankaccount.id AS bankaccount_id, bankaccount.name AS bankaccount_name, bankaccount."IBAN" AS "bankaccount_IBAN", bankaccount."BIC" AS "bankaccount_BIC" 
FROM bankaccount 
WHERE bankaccount.id = ?
2022-02-20 13:56:09,257 INFO sqlalc

This will create a folder named with the inovice number, containing the invoice as an HTML document.

TODO: Invoice styling

![](https://media.gettyimages.com/vectors/under-construction-sign-in-white-background-vector-id536013219?b=1&k=6&m=536013219&s=170x170&h=_-DZGgLDo-84eyZSJZmgwlekma7FYt3LdOdLIC7qktw=)